# Sampling Genome-Scale Models

In this notebook we demonstrate a few tricks to to sampling genome-scale models defined in SBML with maximum efficiency.
To keep the compute times minimal in this demo, we demonstrate the required API calls using e_coli_core.

Note, that we have used code similar to this to sample Recon3D.

In [1]:
import hopsy
from PolyRound.api import PolyRoundApi
import os
import time
import numpy as np
import copy

# Load model

In [2]:
model_path = os.path.join("test_data", "e_coli_core.xml")
polytope = PolyRoundApi.sbml_to_polytope(model_path)
# Note: gurobi is used here. Academic licenses are available for free. If you don't have gurobi, the fallback is glpk. glpk sometimes struggles with numerically challenging models

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-22


# Generate hopsy problem 
Generate the problem object from polytope definition & preprocess by bounding and rounding the polytope.
We add the bounds to ensure the uniform distribution is well-defined on the polytope and we round to increase sampling efficiency.

In [3]:
problem = hopsy.Problem(polytope.A, polytope.b)
problem = hopsy.add_box_constraints(problem, upper_bound=10_000, lower_bound=-10_000, simplify=True)
start = time.perf_counter()
problem = hopsy.round(problem)
print("Computing rounding transformation took", time.perf_counter()-start,"seconds")

Computing rounding transformation took 9.300000169001578 seconds


In [4]:
print(polytope.A.shape)
print(polytope.b.shape)
print(problem.A.shape)
print(problem.b.shape)

(190, 95)
(190,)
(190, 95)
(190,)


# Setup markov chains and random number generators
We require to manually specify a seed, because it improves awareness of the seed. Specifically, the seed is required for scientific reproducibility.

In [5]:
seed = 511
chains, rngs = hopsy.setup(problem, seed, n_chains=4)
n_samples = 10_000
# Either use thinning rule, see  10.1371/journal.pcbi.1011378
# or use one-shot transformation (for expert users). We show one-shot transformation at the end.
thinning = int(1./6*problem.transformation.shape[1])

In [6]:
start = time.perf_counter()
accrate, samples = hopsy.sample(chains, rngs, n_samples, thinning=thinning, n_procs=4)
print("sampling with internal trafo took", time.perf_counter()-start,"seconds")
# accrate is 1 for uniform samples with the default chains given by hopsy.setup()

sampling with internal trafo took 0.6584702769978321 seconds


# Evaluate sampe quality
For highest statistical quality, it is advised to check rhat < 1.01 and ess / n_chains > 100 

In [7]:
rhat = np.max(hopsy.rhat(samples))
print("rhat:", rhat)
ess = np.min(hopsy.ess(samples)) / len(chains)
print("ess:", ess)

rhat: 1.0019304367129511
ess: 1426.2729062496408


# Expert mode for speed: one shot backtransform
By postponing the back transformation from rounded space to original space, we can obtain better performance for high-dimensional models

In [8]:
assert problem.transformation is not None

In [9]:
# deep copy enures that we do not edit the original problem
problem2 = copy.deepcopy(problem)
problem2.transformation=None
problem2.shift=None
seed = 512
chains, rngs = hopsy.setup(problem2, seed, n_chains=4)
n_samples = 10_000
# thinning is still advised when hard drive memory is limisted to not to store too many samples 
thinning = int(1*problem.A.shape[1])  
start = time.perf_counter()
accrate, sample_stack = hopsy.sample(chains, rngs, n_samples, thinning=thinning)
print("sampling took", time.perf_counter()-start,"seconds")
# accrate is 1 for uniform samples with the default chains given by hopsy.setup()

print('sample shape', sample_stack.shape)
rhat = np.max(hopsy.rhat(sample_stack))
print("rhat:", rhat)
ess = np.min(hopsy.ess(sample_stack)) / len(chains)
print("ess:", ess)

# transform samples back all at once
shift_t = np.array([problem.shift]).T
start_trafo = time.perf_counter()
full_samples = np.zeros((len(chains), n_samples, sample_stack.shape[2]))
for i in range(len(chains)):
    full_samples[i] = (problem.transformation@sample_stack[i].T).T + np.tile(shift_t, (1, n_samples)).T
    
print("transformation took", time.perf_counter()-start_trafo,"seconds")
print('sample stats are the same (save numerics) before and after the linear transformation:')
rhat = np.max(hopsy.rhat(full_samples))
print("rhat:", rhat)
ess = np.min(hopsy.ess(full_samples)) / len(chains)
print("ess:", ess)

sampling took 1.0086242019970086 seconds
sample shape (4, 10000, 95)
rhat: 1.000227929785174
ess: 9658.117836358155
transformation took 0.019212077000702266 seconds
sample stats are the same (save numerics) before and after the linear transformation:
rhat: 1.0002279346424705
ess: 9658.117836358155
